In [1]:
%load_ext nvcc4jupyter

from nvcc4jupyter import set_defaults
set_defaults(compiler_args='-arch=sm_100a -Xptxas=-v')

Source files will be saved in "/tmp/tmpf7chzbnf".


We are so fucking back, have a sore throat, IM PISSED AND IN NEED TO CODE AND THINK IMMA GO TAKE SOME COUGHT SUPRESSSANT AND BE RIGHT BACK. 

In [2]:
%%cuda

#include<stdio.h>
#include<stdlib.h>
#include<cuda.h> 
#include<cuda_runtime.h> 
#include<cooperative_groups.h> 

namespace cg = cooperative_groups;

constexpr int N = 4096; 
constexpr int block_size = 1024; 
constexpr int N_blocks = 1; 
constexpr int N_iter = N/block_size;

__global__ void three_step_sync(float*A, float*B)
{
  __shared__ float S[block_size]; 
  auto b = cg::this_thread_block(); 
  int t = b.thread_rank();
  float r = 0.0;
  for (int i = 0; i < N_iter; i++)
  {
    //#loads into shared memory
    S[t] = A[i+t]; 
    b.sync(); //#creates a barrier where each of the threads wait until all the other threads are complete
    r = S[t] + 1.0;
    B[i+t] = r;
    b.sync();//#another barrier for each thread to be done writing results into gmem and then 
    //#and only then does the next loop start again
    
  }
}

int main()
{
  float *A_h, *A_d, *B_h, *B_d; 
  size_t size = N*sizeof(float);
  
  cudaHostAlloc(&A_h, size, cudaHostAllocDefault);
  cudaHostAlloc(&B_h, size, cudaHostAllocDefault); 
  
  cudaMalloc(&A_d, size);
  cudaMalloc(&B_d, size);
  
  for (int i = 0; i < N; i++)
  {
    A_h[i] = (float)(i/1000.0);
  }
  
  cudaMemcpy(A_d, A_h, size, cudaMemcpyHostToDevice); 
  
  three_step_sync<<<N_blocks, block_size>>>(A_d, B_d); 
  
  cudaDeviceSynchronize();
  
  cudaMemcpy(B_h, B_d, size, cudaMemcpyDeviceToHost);
  
  for (int i = 0; i < 50; i++)
  {
    printf("input: %f, output: %f \n", A_h[i], B_h[i]);
  }
  
  cudaFree(A_d);
  cudaFree(B_d);
  cudaFreeHost(A_h); 
  cudaFreeHost(B_h);
  
}


input: 0.000000, output: 1.000000 
input: 0.001000, output: 1.001000 
input: 0.002000, output: 1.002000 
input: 0.003000, output: 1.003000 
input: 0.004000, output: 1.004000 
input: 0.005000, output: 1.005000 
input: 0.006000, output: 1.006000 
input: 0.007000, output: 1.007000 
input: 0.008000, output: 1.008000 
input: 0.009000, output: 1.009000 
input: 0.010000, output: 1.010000 
input: 0.011000, output: 1.011000 
input: 0.012000, output: 1.012000 
input: 0.013000, output: 1.013000 
input: 0.014000, output: 1.014000 
input: 0.015000, output: 1.015000 
input: 0.016000, output: 1.016000 
input: 0.017000, output: 1.017000 
input: 0.018000, output: 1.018000 
input: 0.019000, output: 1.019000 
input: 0.020000, output: 1.020000 
input: 0.021000, output: 1.021000 
input: 0.022000, output: 1.022000 
input: 0.023000, output: 1.023000 
input: 0.024000, output: 1.024000 
input: 0.025000, output: 1.025000 
input: 0.026000, output: 1.026000 
input: 0.027000, output: 1.027000 
input: 0.028000, out

In the above, we have a basic 3 step sync pattern, if b is a block barrier,  there is, per thread, work before the wait point, that is, 
memory operations that each thread does independently, then at the wait point, each thread is blocked until the barrier counts down to all threads arriving there, then after the wait point is completed, the memory operations's results is visible to all threads. 

Next we will look at a more complex barrier. 

In [ ]:
%%cuda

#include<stdio.h>
#include<stdlib.h>
#include<cuda.h> 
#include<cuda_runtime.h> 
#include <cuda/barrier>
#include <cooperative_groups.h>

namespace cg = cooperative_groups;
using barrier = cuda::barrier<cuda::thread_scope_block>;

constexpr int N = 4096; 
constexpr int block_size = 1024; 
constexpr int N_blocks = 1; 
constexpr int TM = N/block_size;

__global__ void three_step_sync(float*A, float*B)
{
  __shared__ float S[block_size]; 
  auto b = cg::this_thread_block(); 
  int t = b.thread_rank();
  float r = 0.0;
  __shared__ barrier bar; //#create barrier on shared memory. 
  
  //# To initalize a barrier, we need to make sure the threads are synchronized, and we initalize it with the expected arrival count
  //# any single thread can init this barrier, the arrival count in our case is going to be block size. 
  if (t == 0)
  {
    init(&bar, b.size());

  }
  b.sync(); 
  //# at this point, thread 0 issued the init of the barrier instruction, and hence, things being issued on other threads 
  //# might happen paralell to the completetion of initing the barrier, therefore our threads are out of sync, and its hard for us to tell
  //# when all the threads will arive at the barrier, therefore, we use b.sync() and just get all the threads in line :)
  
  
  

}

int main()
{
  float *A_h, *A_d, *B_h, *B_d; 
  size_t size = N*sizeof(float);
  
  cudaHostAlloc(&A_h, size, cudaHostAllocDefault);
  cudaHostAlloc(&B_h, size, cudaHostAllocDefault); 
  
  cudaMalloc(&A_d, size);
  cudaMalloc(&B_d, size);
  
  for (int i = 0; i < N; i++)
  {
    A_h[i] = (float)(i/1000.0);
  }
  
  cudaMemcpy(A_d, A_h, size, cudaMemcpyHostToDevice); 
  
  three_step_sync<<<N_blocks, block_size>>>(A_d, B_d); 
  
  cudaDeviceSynchronize();
  
  cudaMemcpy(B_h, B_d, size, cudaMemcpyDeviceToHost);
  
  for (int i = 0; i < 50; i++)
  {
    printf("input: %f, output: %f \n", A_h[i], B_h[i]);
  }
  
  cudaFree(A_d);
  cudaFree(B_d);
  cudaFreeHost(A_h); 
  cudaFreeHost(B_h);
  
}

input: 0.000000, output: 0.000000 
input: 0.001000, output: 0.000000 
input: 0.002000, output: 0.000000 
input: 0.003000, output: 0.000000 
input: 0.004000, output: 0.000000 
input: 0.005000, output: 0.000000 
input: 0.006000, output: 0.000000 
input: 0.007000, output: 0.000000 
input: 0.008000, output: 0.000000 
input: 0.009000, output: 0.000000 
input: 0.010000, output: 0.000000 
input: 0.011000, output: 0.000000 
input: 0.012000, output: 0.000000 
input: 0.013000, output: 0.000000 
input: 0.014000, output: 0.000000 
input: 0.015000, output: 0.000000 
input: 0.016000, output: 0.000000 
input: 0.017000, output: 0.000000 
input: 0.018000, output: 0.000000 
input: 0.019000, output: 0.000000 
input: 0.020000, output: 0.000000 
input: 0.021000, output: 0.000000 
input: 0.022000, output: 0.000000 
input: 0.023000, output: 0.000000 
input: 0.024000, output: 0.000000 
input: 0.025000, output: 0.000000 
input: 0.026000, output: 0.000000 
input: 0.027000, output: 0.000000 
input: 0.028000, out